In [1]:
from sklearn.neural_network import MLPClassifier as mlp
import sys
sys.path.append("/home/watkinsz/Desktop/For_Zack/ML")
sys.path.append("/home/watkinsz/Desktop/For_Zack/ML/FeH")
from fitsParser.fitsParser import fitsParser
import pandas as pd
import numpy as np
import re
from numpy import linspace
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from modelValidator import psMetric, cut, shuffleDwarfs, plotRes, plot, norm_inputs, monte_carlo, printTable
from sklearn.gaussian_process import GaussianProcessRegressor as gpr
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from function import save_data
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Nadam


fits = fitsParser(includefile = 'fitsSetup.txt')
fits.data['u']+=0.4 #compensate for percent level offset in ab magnitudes from sdss
fits.makeColours(N=10)
colours = [ 'g-i', 'CaHK-g', 'u-g', 'CaHK-g', 'CaHK-i', 'z-y']
test_colours = ['pristine_col'+x for x in '1234']+['g-i', 'g-r']
def moreCols(fits):
    fits.data['pristine_col4'] = fits.data['u-g']
    fits.data['pristine_col1'] = fits.data['CaHK-g']
    fits.data['pristine_col2'] = fits.data['g-i']
    fits.data['pristine_col3'] = fits.data['u-CaHK']
moreCols(fits)
sdss = fitsParser(includefile = 'FeHdata.txt')
colours = ['pristine_col'+x for x in '1']+['g-r']
fits.data['FeHadop'] = fits.data['FeH']
#colours = fits.colours

data = pd.DataFrame(fits.data)
data = data.sample(frac = 1).reset_index(drop= True)
data['test_flag'] = [0]*int(len(data)*0.8)+[1]*(len(data)-int(len(data)*0.8))

def getMags(colour):
    return colour[:colour.index('-')], colour[colour.index('-')+1:]
pre_transform = lambda l, bins : map(lambda x : 0 if x>=0 else   x + min([c-x for c in bins if x<=c]) , l)

sdss = fitsParser(includefile = 'FeHdata.txt')
youc = sdss.getAllObj()[1]
cut(youc, metThresh = 0)
youc.makeColours(N=10)
conversion = {
     'u_0' : 'u',
    'CaHK_0' : 'CaHK' 
}
conversion.update({x+'_fpsf' : x for x in 'grizy'})
for col in youc.colours:
    mag1, mag2 = getMags(col)
    youc.data[conversion[mag1]+'-'+conversion[mag2]] = youc.data[col]

for field in youc.data.keys():
    if field in conversion:
        youc.data[conversion[field]]= youc.data[field]
youc.data = youc.data.sample(frac=1)
moreCols(youc)
youc.data['FeHadop'] = pre_transform(youc.data['FeHadop'], [ 0,-0.5,-1,-1.5,-2,-3,-4])
youc.data['FeHround'] = youc.data['FeHadop']
youc.data['test_flag'] = 1
classes = [ 0,-0.5,-1,-1.5,-2,-3,-4]       

def baseModel(inputsDim, outDim, Nlayers=30, Nodes = 128):
    model = Sequential()
    model.add(Dense(Nodes, input_dim = inputsDim, activation = 'relu'))
    
    for i in range(Nlayers-1):
        model.add(Dense(Nodes, activation = 'relu'))
    model.add(Dense(outDim, activation = 'softmax'))
    op = Nadam(lr = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer=op, metrics=['accuracy'])
    return model


Using TensorFlow backend.



Data saved

condition on dg_fpsf cut 0 inputs
condition on dr_fpsf cut 0 inputs
condition on di_fpsf cut 0 inputs
condition on dz_fpsf cut 0 inputs
condition on dy_fpsf cut 0 inputs
condition on dCaHK_0 cut 0 inputs
condition on du_0 cut 0 inputs
condition on dFeHadop cut 0 inputs
0.57% of inputs cut


In [2]:
class transformer_man:
    
    def __init__(self, classes, func, inverse_func):
        self.classes = classes
        self.func = func
        self.inverse_func = inverse_func

    def transform(self,data):
        return [[1 if self.func(x, c) else 0 for c in self.classes] for x in data]
    
    def inverse_transform(self, trans_data):
        return [self.inverse_func(x) for x in trans_data]



In [3]:
transformer = transformer_man(classes,
    lambda x, c :  x == c, 
    lambda x : classes[x.index(1)])
data['label'] = transformer.transform(data['FeHadop'])
train = data[data['test_flag']==0]

model = baseModel(len(colours), len(classes), Nlayers = 12, Nodes = 64)
params = {
    'x' : train[colours].values,
    'y' : np.array([x for x in train['label'].values]),
    'verbose' : 2,
    'batch_size': 100,
    'epochs' : 200,
    'validation_data' : 
    (data[data['test_flag']==1][colours].values, np.array([x for x in data[data['test_flag']==1]['label'].values]))
}

model.fit(**params)

Train on 22163 samples, validate on 5541 samples
Epoch 1/200
 - 3s - loss: 1.6104 - acc: 0.2634 - val_loss: 1.4472 - val_acc: 0.3683
Epoch 2/200
 - 1s - loss: 1.4013 - acc: 0.3611 - val_loss: 1.4352 - val_acc: 0.3730
Epoch 3/200
 - 1s - loss: 1.3614 - acc: 0.3789 - val_loss: 1.3476 - val_acc: 0.3898
Epoch 4/200
 - 1s - loss: 1.3334 - acc: 0.3901 - val_loss: 1.3569 - val_acc: 0.3770
Epoch 5/200
 - 1s - loss: 1.3186 - acc: 0.3971 - val_loss: 1.4912 - val_acc: 0.3140
Epoch 6/200
 - 1s - loss: 1.3172 - acc: 0.3976 - val_loss: 1.3431 - val_acc: 0.3967
Epoch 7/200
 - 1s - loss: 1.3083 - acc: 0.3981 - val_loss: 1.3419 - val_acc: 0.3936
Epoch 8/200
 - 1s - loss: 1.3041 - acc: 0.4040 - val_loss: 1.3139 - val_acc: 0.4017
Epoch 9/200
 - 1s - loss: 1.3002 - acc: 0.4087 - val_loss: 1.3070 - val_acc: 0.4062
Epoch 10/200
 - 1s - loss: 1.2941 - acc: 0.4081 - val_loss: 1.3194 - val_acc: 0.4061
Epoch 11/200
 - 1s - loss: 1.2886 - acc: 0.4064 - val_loss: 1.3045 - val_acc: 0.4010
Epoch 12/200
 - 1s - loss

Epoch 97/200
 - 1s - loss: 1.2296 - acc: 0.4347 - val_loss: 1.2655 - val_acc: 0.4248
Epoch 98/200
 - 1s - loss: 1.2311 - acc: 0.4376 - val_loss: 1.2592 - val_acc: 0.4212
Epoch 99/200
 - 1s - loss: 1.2257 - acc: 0.4362 - val_loss: 1.2557 - val_acc: 0.4216
Epoch 100/200
 - 1s - loss: 1.2255 - acc: 0.4318 - val_loss: 1.3118 - val_acc: 0.4088
Epoch 101/200
 - 1s - loss: 1.2323 - acc: 0.4312 - val_loss: 1.2743 - val_acc: 0.4272
Epoch 102/200
 - 1s - loss: 1.2307 - acc: 0.4338 - val_loss: 1.2707 - val_acc: 0.4122
Epoch 103/200
 - 1s - loss: 1.2235 - acc: 0.4380 - val_loss: 1.2627 - val_acc: 0.4203
Epoch 104/200
 - 1s - loss: 1.2272 - acc: 0.4340 - val_loss: 1.2663 - val_acc: 0.4277
Epoch 105/200
 - 1s - loss: 1.2262 - acc: 0.4378 - val_loss: 1.2830 - val_acc: 0.4209
Epoch 106/200
 - 1s - loss: 1.2256 - acc: 0.4348 - val_loss: 1.2605 - val_acc: 0.4324
Epoch 107/200
 - 1s - loss: 1.2269 - acc: 0.4386 - val_loss: 1.2865 - val_acc: 0.3967
Epoch 108/200
 - 1s - loss: 1.2287 - acc: 0.4325 - val_lo

Epoch 193/200
 - 1s - loss: 1.2096 - acc: 0.4467 - val_loss: 1.2580 - val_acc: 0.4293
Epoch 194/200
 - 1s - loss: 1.2116 - acc: 0.4450 - val_loss: 1.2684 - val_acc: 0.4254
Epoch 195/200
 - 1s - loss: 1.2125 - acc: 0.4485 - val_loss: 1.2642 - val_acc: 0.4169
Epoch 196/200
 - 1s - loss: 1.2117 - acc: 0.4452 - val_loss: 1.2664 - val_acc: 0.4290
Epoch 197/200
 - 1s - loss: 1.2111 - acc: 0.4441 - val_loss: 1.2595 - val_acc: 0.4357
Epoch 198/200
 - 1s - loss: 1.2079 - acc: 0.4446 - val_loss: 1.2557 - val_acc: 0.4378
Epoch 199/200
 - 1s - loss: 1.2069 - acc: 0.4472 - val_loss: 1.2613 - val_acc: 0.4326
Epoch 200/200
 - 1s - loss: 1.2065 - acc: 0.4438 - val_loss: 1.2615 - val_acc: 0.4342


In [4]:
pretrans = lambda data : [[1 if False not in [x[i] > y or y == x[i] for y in x ] else 0 for i in range(len(x))] for x in data] 
pretrans2 = lambda data : [[0]*4+[0,0,1] if x[2]>0.6 else [0]*4+[0,1,0] if x[1]>0.33 else [0]*4+[1,0,0] for x in data]

z = model.predict(data[colours])
data['FeH_pred'] = transformer.inverse_transform(pretrans(z))
data['FeHround'] = data['FeHadop'] 
i = list(classes)

def conf_matrix(data, classes):
    return [[len(data[(data['FeHround']==c1)&(data['FeH_pred']==c2)])*1.0/len(data[(data['FeHround']==c1)])
             if len(data[(data['FeHround']==c1)])!= 0 else -1
             for c2 in classes] for c1 in classes]

pur, com = psMetric(data, inc = i)
print printTable(i ,pur,com)
print pd.DataFrame(conf_matrix(data, classes), columns = classes, index = classes) , '\n'      
             
pur2, com2 = psMetric(data[data['test_flag']==1], inc = i)
print printTable(i,pur2,com2)
print pd.DataFrame(conf_matrix(data[data['test_flag']==1], classes), columns = classes, index = classes),'\n'

| [Fe/H]< | purity | completness |
----------------------------------
| 0.0     | 0.9402 | 1.0000      |
| -0.5    | 0.9228 | 0.7739      |
| -1.0    | 0.8931 | 0.7355      |
| -1.5    | 0.8169 | 0.7393      |
| -2.0    | 0.7327 | 0.7006      |
| -3.0    | 0.6284 | 0.6733      |
| -4.0    | 0.3732 | 0.3539      |

           0.0      -0.5      -1.0      -1.5      -2.0      -3.0      -4.0
 0.0  0.757774  0.129998  0.027122  0.020108  0.017068  0.032499  0.015431
-0.5  0.510042  0.298926  0.070061  0.044839  0.025689  0.036665  0.013779
-1.0  0.300233  0.212326  0.228605  0.160000  0.042326  0.043953  0.012558
-1.5  0.160640  0.141168  0.135837  0.312703  0.165508  0.071164  0.012981
-2.0  0.094208  0.067225  0.077925  0.217958  0.388230  0.141893  0.012561
-3.0  0.054963  0.016542  0.021612  0.036286  0.195304  0.647279  0.028015
-4.0  0.172705  0.016908  0.018116  0.041063  0.086957  0.310386  0.353865 

| [Fe/H]< | purity | completness |
----------------------------------
| 0.0     | 

In [5]:

z = model.predict(youc.data[colours])
youc.data['FeH_pred'] = transformer.inverse_transform(pretrans2(z))
youc.data['FeHadop'] = youc.data['FeHround']
i = list(classes)

def conf_matrix(data, classes):
    return [[len(data[(data['FeHround']==c1)&(data['FeH_pred']==c2)])*1.0/len(data[(data['FeHround']==c1)])
             if len(data[(data['FeHround']==c1)])!= 0 else -1
             for c2 in classes] for c1 in classes]

pur, com = psMetric(youc.data, inc = i)
print printTable(i ,pur,com)
print pd.DataFrame(conf_matrix(youc.data, classes), columns = classes, index = classes)   , '\n'          
             
save_data(youc.data, 'youcout.asc')

| [Fe/H]< | purity | completness |
----------------------------------
| 0.0     | 1.0000 | 1.0000      |
| -0.5    | 1.0000 | 1.0000      |
| -1.0    | 1.0000 | 1.0000      |
| -1.5    | 1.0000 | 1.0000      |
| -2.0    | 1.0000 | 1.0000      |
| -3.0    | 0.0000 | 0.0000      |
| -4.0    | 0.0000 | 0.0000      |

       0.0  -0.5  -1.0  -1.5      -2.0      -3.0      -4.0
 0.0  -1.0  -1.0  -1.0  -1.0 -1.000000 -1.000000 -1.000000
-0.5  -1.0  -1.0  -1.0  -1.0 -1.000000 -1.000000 -1.000000
-1.0  -1.0  -1.0  -1.0  -1.0 -1.000000 -1.000000 -1.000000
-1.5  -1.0  -1.0  -1.0  -1.0 -1.000000 -1.000000 -1.000000
-2.0   0.0   0.0   0.0   0.0  0.967593  0.021605  0.010802
-3.0   0.0   0.0   0.0   0.0  1.000000  0.000000  0.000000
-4.0   0.0   0.0   0.0   0.0  1.000000  0.000000  0.000000 


Data saved

